# Lab3

In [54]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate, cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [2]:

##2 - 0.42; 3 - 0.40; 7 - 0.10; 8 - 0.32; 9 - 0.28;

features = []
gender = []
fs_f_g = []
i_f_g = []
t_lda_f_g = []
t_liwc_f_g = []
t_h_f_g = []
fs_f_a = []
i_f_a = []
t_lda_f_a = []
t_liwc_f_a = []
t_h_f_a = []
fs_f_ga = []
i_f_ga = []
t_lda_f_ga = []
t_liwc_f_ga = []
t_h_f_ga = []
fs_f = []
i_f = []
t_lda_f = []
t_liwc_f = []
t_h_f = []
age = []
group_age = []
c = 0
labels = []
edu = []
work = []
edul = []
occ = []

## Loading features

In [6]:
for city in ['London', 'Singapore', 'NewYork']:
    c_fs_f = pd.read_csv('Features/features' + city + '/Foursquare/venueCategoriesLDA6Features.csv', header=0)
    c_i_f = pd.read_csv('Features/features' + city + '/Instagram/imageConceptsFeatures.csv', header=0)
    c_t_lda_f = pd.read_csv('Features/features' + city + '/Twitter/LDA50Features.csv', header=0)
    c_t_liwc_f = pd.read_csv('Features/features' + city + '/Twitter/LIWCFeatures.csv', header=0)
    c_t_h_f = pd.read_csv('Features/features' + city + '/Twitter/manuallyDefinedTextFeatures.csv', header=0)

    #download set of example
    dp = pd.read_csv(city + 'Train.csv', header=0)
    edu = edu + dp['educationInfoForAgeGroupEstimation'].values.tolist()
    work += dp['workInfoForAgeGroupEstimation'].values.tolist()
    edul += dp['educationLevel'].values.tolist()
    occ += dp['occupation'].values.tolist()
    found = 0
    notfound = 0
    for i in dp.index:
        f = []
        fs = c_fs_f.query('_id=="' + dp['row ID'][i] + '"').values.tolist()
        if len(fs) == 0:
            fs = [0 for i in range(6)]
        else:
            fs = fs[0][1:]
        ins = c_i_f.query('_id=="' + dp['row ID'][i] + '"').values.tolist()
        if len(ins) == 0:
            ins = [0 for i in range(1001)]
        else:
            ins = ins[0][1:]
        t_lda = c_t_lda_f.query('_id=="' + dp['row ID'][i] + '"').values.tolist()
        if len(t_lda) == 0:
            t_lda = [0 for i in range(50)]
        else:
            t_lda = t_lda[0][1:]
        t_liwc = c_t_liwc_f.query('_id=="' + dp['row ID'][i] + '"').values.tolist()
        if len(t_liwc) == 0:
            t_liwc= [0 for i in range(70)]
        else:
            t_liwc = t_liwc[0][1:]
        t_h = c_t_h_f.query('_id=="' + dp['row ID'][i] + '"').values.tolist()
        if len(t_h) == 0:
            t_h = [0 for i in range(14)]
        else:
            t_h = t_h[0][1:]

        #features.append(fs + ins + t_lda + t_liwc + t_h + [c])
        t_lda_f.append(t_lda + [c])
        t_liwc_f.append(t_liwc + [c])
        fs_f.append(fs + [c])
        i_f.append(ins + [c])
        t_h_f.append(t_h + [c])
        if (dp['gender'][i] != None):
            t_lda_f_g.append(t_lda + [c])
            t_liwc_f_g.append(t_liwc + [c])
            fs_f_g.append(fs + [c])
            i_f_g.append(ins + [c])
            t_h_f_g.append(t_h + [c])
            gender.append(dp['gender'][i])
        if (dp['realAge'][i] != None):
            #print('without age')
            t_lda_f_a.append(t_lda + [c])
            t_liwc_f_a.append(t_liwc + [c])
            fs_f_a.append(fs + [c])
            i_f_a.append(ins + [c])
            t_h_f_a.append(t_h + [c])
            age.append(dp['gender'][i])
        if (dp['ageGroup'][i] != None):
            t_lda_f_ga.append(t_lda + [c])
            t_liwc_f_ga.append(t_liwc + [c])
            fs_f_ga.append(fs + [c])
            i_f_ga.append(ins + [c])
            t_h_f_ga.append(t_h + [c])
            group_age.append(dp['gender'][i])
    c += 1


## Encoding features

In [7]:
gender_encode = LabelEncoder()
gender_feature = gender_encode.fit_transform(gender)
age_encode = LabelEncoder()
age_feature = age_encode.fit_transform(age)
group_age_encode = LabelEncoder()
group_age_feature = group_age_encode.fit_transform(group_age)
edu_encode = LabelEncoder()
work_encode = LabelEncoder()
edul_encode = LabelEncoder()
occ_encode = LabelEncoder()
edu = edu_encode.fit_transform(edu)
work = work_encode.fit_transform(work)
edul = edul_encode.fit_transform(edul)
occ = occ_encode.fit_transform(occ)
labels = [[e1, e2, e3, e4] for (e1, e2, e3, e4) in zip(edu, work, edul, occ)]


## Learning algorithms

### Gender

In [8]:
gender_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced')
t_liwc_gender_scores = cross_val_score(gender_predict, t_liwc_f_g, gender_feature, cv=10, scoring='f1_macro').mean()
print(t_liwc_gender_scores)
gender_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced')
fs_gender_scores = cross_val_score(gender_predict, fs_f_g, gender_feature, cv=10, scoring='f1_macro').mean()
print(fs_gender_scores)
gender_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced')
t_lda_gender_scores = cross_val_score(gender_predict, t_lda_f_g, gender_feature, cv=10, scoring='f1_macro').mean()
print(t_lda_gender_scores)
gender_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced')
i_gender_scores = cross_val_score(gender_predict, i_f_g, gender_feature, cv=10, scoring='f1_macro').mean()
print(i_gender_scores)
gender_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced')
t_h_gender_scores = cross_val_score(gender_predict, t_h_f_g, gender_feature, cv=10, scoring='f1_macro').mean()
print(t_h_gender_scores)

0.7801505569404696
0.659878678509249
0.8043081479053835
0.7286206170994275
0.8015031001664992


### Age

In [9]:
age_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced')
t_liwc_age_scores = cross_val_score(age_predict, t_liwc_f_a, age_feature, cv=10, scoring='f1_macro').mean().mean()
print(t_liwc_age_scores)
age_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced')
fs_age_scores = cross_val_score(age_predict, fs_f_a, age_feature, cv=10, scoring='f1_macro').mean()
print(fs_age_scores)
age_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced')
t_lda_age_scores = cross_val_score(age_predict, t_lda_f_a, age_feature, cv=10, scoring='f1_macro').mean()
print(t_lda_age_scores)
age_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced')
i_age_scores = cross_val_score(age_predict, i_f_a, age_feature, cv=10, scoring='f1_macro').mean()
print(i_age_scores)
age_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced')
t_h_age_scores = cross_val_score(age_predict, t_h_f_a, age_feature, cv=10, scoring='f1_macro').mean()
print(t_h_age_scores)


0.7778442524977787
0.6593274165812081
0.8175737660478981
0.7283135192700665
0.8018091776100599


### Group age

In [10]:
group_age_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced')
t_liwc_group_age_scores = cross_val_score(group_age_predict, t_liwc_f_ga, group_age_feature, cv=10, scoring='f1_macro').mean()
print(t_liwc_group_age_scores)
group_age_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced')
fs_group_age_scores = cross_val_score(group_age_predict, fs_f_ga, group_age_feature, cv=10, scoring='f1_macro').mean()
print(fs_group_age_scores)
group_age_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced')
t_lda_group_age_scores = cross_val_score(group_age_predict, t_lda_f_ga, group_age_feature, cv=10, scoring='f1_macro').mean()
print(t_lda_group_age_scores)
group_age_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced')
i_group_age_scores = cross_val_score(group_age_predict, i_f_ga, group_age_feature, cv=10, scoring='f1_macro').mean()
print(i_group_age_scores)
group_age_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced')
t_h_group_age_scores = cross_val_score(group_age_predict, t_h_f_ga, group_age_feature, cv=10, scoring='f1_macro').mean()
print(t_h_group_age_scores)


0.7730603911674443
0.6541819323151061
0.8149102934654098
0.7077252235081344
0.7966369532119568


In [11]:
early_predict = RandomForestClassifier()
early_scores = cross_val_score(early_predict,
                               [t_liwc_f_g[i] + t_lda_f_g[i] + t_h_f_g[i] + fs_f_g[i] + i_f_g[i] for i in range(len(gender))],
                               gender_feature, cv=10, scoring='f1_macro').mean()
print(early_scores)

0.8513930446975817


In [12]:
early_predict = RandomForestClassifier()
early_scores = cross_val_score(early_predict,
                               [t_liwc_f_a[i] + t_lda_f_a[i] + t_h_f_a[i] + fs_f_a[i] + i_f_a[i] for i in range(len(age))],
                               age_feature, cv=10, scoring='f1_macro').mean()
print(early_scores)

0.8512808491843418


In [13]:
early_predict = RandomForestClassifier()
early_scores = cross_val_score(early_predict,
                               [t_liwc_f_ga[i] + t_lda_f_ga[i] + t_h_f_ga[i] + fs_f_ga[i] + i_f_ga[i] for i in range(len(group_age))],
                               group_age_feature, cv=10, scoring='f1_macro').mean()
print(early_scores)

0.8513284666703012


## KFold for Gender

In [14]:
kf = KFold(n_splits=10)
gender_ind = range(len(gender_feature))
avg_f1 = 0

In [15]:
for train_ind, test_ind in kf.split(gender_ind, gender_feature):
    gender_train = [gender_feature[i] for i in train_ind]
    gender_test = [gender_feature[i] for i in test_ind]
    train_s_ind, train_f_ind, gender_s_ind, gender_f_ind = train_test_split(train_ind, gender_train, test_size=0.7,stratify=gender_train,random_state=1)
    t_liwc_gender_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced')
    t_liwc_gender_predict.fit([t_liwc_f_g[j] for j in train_s_ind], gender_s_ind)
    train = t_liwc_gender_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
    test = t_liwc_gender_predict.predict_proba([t_liwc_f_g[j] for j in test_ind])
    fs_gender_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced')
    fs_gender_predict.fit([fs_f_g[j] for j in train_s_ind], gender_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, fs_gender_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, fs_gender_predict.predict_proba([fs_f_g[j] for j in test_ind]))]
    t_lda_gender_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced')
    t_lda_gender_predict.fit([t_lda_f_g[j] for j in train_s_ind], gender_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, t_lda_gender_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, t_lda_gender_predict.predict_proba([t_lda_f_g[j] for j in test_ind]))]
    i_gender_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced')
    i_gender_predict.fit([i_f_g[j] for j in train_s_ind], gender_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, i_gender_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, i_gender_predict.predict_proba([i_f_g[j] for j in test_ind]))]
    t_h_gender_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced')
    t_h_gender_predict.fit([t_h_f_g[j] for j in train_s_ind], gender_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, t_h_gender_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, t_h_gender_predict.predict_proba([t_h_f_g[j] for j in test_ind]))]

    best_f1 = 0
    for i in range(50, 155, 5) :
      gender_a_predict = RandomForestClassifier(n_estimators=i)
      f1 = cross_val_score(gender_a_predict, train + test, gender_f_ind + gender_test, scoring='f1_macro', cv=10).mean()
      if (f1 > best_f1):
        best_f1 = f1
    #gender_a_predict.fit(train, gender_f_ind)
    #gender_pred = gender_a_predict.predict(test)
    avg_f1 += best_f1

avg_f1 /= 10
print(avg_f1)

0.7252993841872477


## KFold for Age

In [16]:
kf = KFold(n_splits=10)
age_ind = range(len(age_feature))
avg_f1 = 0

In [17]:
for train_ind, test_ind in kf.split(age_ind, age_feature):
    age_train = [age_feature[i] for i in train_ind]
    age_test = [age_feature[i] for i in test_ind]
    train_s_ind, train_f_ind, age_s_ind, age_f_ind = train_test_split(train_ind, age_train, test_size=0.7,
                                                                            stratify=age_train, random_state=1)
    age_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced')
    age_predict.fit([t_liwc_f_g[j] for j in train_s_ind], age_s_ind)
    train = age_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
    test = age_predict.predict_proba([t_liwc_f_g[j] for j in test_ind])
    age_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced')
    age_predict.fit([fs_f_g[j] for j in train_s_ind], age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, age_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, age_predict.predict_proba([fs_f_g[j] for j in test_ind]))]
    age_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced')
    age_predict.fit([t_lda_f_g[j] for j in train_s_ind], age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, age_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, age_predict.predict_proba([t_lda_f_g[j] for j in test_ind]))]
    age_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced')
    age_predict.fit([i_f_g[j] for j in train_s_ind], age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, age_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, age_predict.predict_proba([i_f_g[j] for j in test_ind]))]
    age_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced')
    age_predict.fit([t_h_f_g[j] for j in train_s_ind], age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, age_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, age_predict.predict_proba([t_h_f_g[j] for j in test_ind]))]

    best_f1 = 0
    for i in range(50, 155, 5) :
      age_a_predict = RandomForestClassifier(n_estimators=i)
      f1 = cross_val_score(age_a_predict, train + test, age_f_ind + age_test, scoring='f1_macro', cv=10).mean()
      if (f1 > best_f1):
        best_f1 = f1
    #gender_a_predict.fit(train, gender_f_ind)
    #gender_pred = gender_a_predict.predict(test)
    avg_f1 += best_f1

avg_f1 /= 10
print(avg_f1)

0.7227534419939464


## KFold for Group age

In [18]:
kf = KFold(n_splits=10)
group_age_ind = range(len(group_age_feature))
avg_f1 = 0

In [19]:
for train_ind, test_ind in kf.split(group_age_ind, group_age_feature):
    group_age_train = [group_age_feature[i] for i in train_ind]
    group_age_test = [group_age_feature[i] for i in test_ind]
    train_s_ind, train_f_ind, group_age_s_ind, group_age_f_ind = train_test_split(train_ind, group_age_train, test_size=0.7,
                                                                            stratify=group_age_train, random_state=1)
    group_age_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced', max_depth=10)
    group_age_predict.fit([t_liwc_f_g[j] for j in train_s_ind], group_age_s_ind)
    train = group_age_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
    test = group_age_predict.predict_proba([t_liwc_f_g[j] for j in test_ind])
    group_age_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced', max_depth=10)
    group_age_predict.fit([fs_f_g[j] for j in train_s_ind], group_age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, group_age_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, group_age_predict.predict_proba([fs_f_g[j] for j in test_ind]))]
    group_age_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced', max_depth=10)
    group_age_predict.fit([t_lda_f_g[j] for j in train_s_ind], group_age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, group_age_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, group_age_predict.predict_proba([t_lda_f_g[j] for j in test_ind]))]
    group_age_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced', max_depth=10)
    group_age_predict.fit([i_f_g[j] for j in train_s_ind], group_age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, group_age_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, group_age_predict.predict_proba([i_f_g[j] for j in test_ind]))]
    group_age_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced', max_depth=10)
    group_age_predict.fit([t_h_f_g[j] for j in train_s_ind], group_age_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, group_age_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, group_age_predict.predict_proba([t_h_f_g[j] for j in test_ind]))]

    best_f1 = 0
    for i in range(50, 155, 5) :
      group_age_a_predict = RandomForestClassifier(n_estimators=i)
      f1 = cross_val_score(group_age_a_predict, train + test, group_age_f_ind + group_age_test, scoring='f1_macro', cv=10).mean()
      if (f1 > best_f1):
        best_f1 = f1
    #gender_a_predict.fit(train, gender_f_ind)
    #gender_pred = gender_a_predict.predict(test)
    avg_f1 += best_f1

avg_f1 /= 10
print(avg_f1)

0.7010729536193081


## Splitting data on train/test and learning RF (Gender)

In [20]:
train_s_ind, train_f_ind, gender_s_ind, gender_f_ind = train_test_split(gender_ind, gender_feature, test_size=0.7,stratify=gender_feature,random_state=1)
t_liwc_gender_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced', max_depth=10)
t_liwc_gender_predict.fit([t_liwc_f_g[j] for j in train_s_ind], gender_s_ind)
train = t_liwc_gender_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
t_liwc_gender = t_liwc_gender_predict.predict(t_liwc_f)
t_liwc_gender_proba = t_liwc_gender_predict.predict_proba(t_liwc_f)
fs_gender_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced', max_depth=10)
fs_gender_predict.fit([fs_f_g[j] for j in train_s_ind], gender_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, fs_gender_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
fs_gender = fs_gender_predict.predict(fs_f)
fs_gender_proba = fs_gender_predict.predict_proba(fs_f)
t_lda_gender_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced', max_depth=10)
t_lda_gender_predict.fit([t_lda_f_g[j] for j in train_s_ind], gender_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, t_lda_gender_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
t_lda_gender = t_lda_gender_predict.predict(t_lda_f)
t_lda_gender_proba = t_lda_gender_predict.predict_proba(t_lda_f)
i_gender_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced', max_depth=10)
i_gender_predict.fit([i_f_g[j] for j in train_s_ind], gender_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, i_gender_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
i_gender = i_gender_predict.predict(i_f)
i_gender_proba = i_gender_predict.predict_proba(i_f)
t_h_gender_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced', max_depth=10)
t_h_gender_predict.fit([t_h_f_g[j] for j in train_s_ind], gender_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, t_h_gender_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
t_h_gender = t_h_gender_predict.predict(t_h_f)
t_h_gender_proba = t_h_gender_predict.predict_proba(t_h_f)


In [21]:
gender_a_predict = RandomForestClassifier(max_depth=10)
gender_a_predict.fit(train, gender_f_ind)
gender_a = gender_a_predict.predict([l1 + l2 + l3 + l4 + l5 for (l1, l2, l3, l4, l5) in zip(t_liwc_gender_proba,
                                                                                            fs_gender_proba,
                                                                                            t_lda_gender_proba,
                                                                                            i_gender_proba,
                                                                                            t_h_gender_proba)])
gender_a_proba = gender_a_predict.predict_proba([l1 + l2 + l3 + l4 + l5 for (l1, l2, l3, l4, l5) in zip(t_liwc_gender_proba,
                                                                                            fs_gender_proba,
                                                                                            t_lda_gender_proba,
                                                                                            i_gender_proba,
                                                                                            t_h_gender_proba)])


## Splitting data on train/test and learning RF (Age)

In [22]:
train_s_ind, train_f_ind, age_s_ind, age_f_ind = train_test_split(age_ind, age_feature, test_size=0.7,stratify=age_feature,random_state=1)
t_liwc_age_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced', max_depth=10)
t_liwc_age_predict.fit([t_liwc_f_g[j] for j in train_s_ind], age_s_ind)
train = t_liwc_age_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
t_liwc_age = t_liwc_age_predict.predict(t_liwc_f)
t_liwc_age_proba = t_liwc_age_predict.predict_proba(t_liwc_f)
fs_age_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced', max_depth=10)
fs_age_predict.fit([fs_f_g[j] for j in train_s_ind], age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, fs_age_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
fs_age = fs_age_predict.predict(fs_f)
fs_age_proba = fs_age_predict.predict_proba(fs_f)
t_lda_age_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced', max_depth=10)
t_lda_age_predict.fit([t_lda_f_g[j] for j in train_s_ind], age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, t_lda_age_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
t_lda_age = t_lda_age_predict.predict(t_lda_f)
t_lda_age_proba = t_lda_age_predict.predict_proba(t_lda_f)
i_age_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced', max_depth=10)
i_age_predict.fit([i_f_g[j] for j in train_s_ind], age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, i_age_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
i_age = i_age_predict.predict(i_f)
i_age_proba = i_age_predict.predict_proba(i_f)
t_h_age_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced', max_depth=10)
t_h_age_predict.fit([t_h_f_g[j] for j in train_s_ind], age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, t_h_age_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
t_h_age = t_h_age_predict.predict(t_h_f)
t_h_age_proba = t_h_age_predict.predict_proba(t_h_f)


In [23]:
age_a_predict = RandomForestClassifier(max_depth=10)
age_a_predict.fit(train, age_f_ind)
age_a = age_a_predict.predict([l1 + l2 + l3 + l4 + l5 for (l1, l2, l3, l4, l5) in zip(t_liwc_age_proba,
                                                                                            fs_age_proba,
                                                                                            t_lda_age_proba,
                                                                                            i_age_proba,
                                                                                            t_h_age_proba)])
age_a_proba = age_a_predict.predict_proba([l1 + l2 + l3 + l4 + l5 for (l1, l2, l3, l4, l5) in zip(t_liwc_age_proba,
                                                                                            fs_age_proba,
                                                                                            t_lda_age_proba,
                                                                                            i_age_proba,
                                                                                            t_h_age_proba)])


## Splitting data on train/test and learning RF (Group age)

In [24]:
train_s_ind, train_f_ind, group_age_s_ind, group_age_f_ind = train_test_split(group_age_ind, group_age_feature, test_size=0.7,stratify=group_age_feature,random_state=1)
t_liwc_group_age_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced', max_depth=10)
t_liwc_group_age_predict.fit([t_liwc_f_g[j] for j in train_s_ind], group_age_s_ind)
train = t_liwc_group_age_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
t_liwc_group_age = t_liwc_group_age_predict.predict(t_liwc_f)
t_liwc_group_age_proba = t_liwc_group_age_predict.predict_proba(t_liwc_f)
fs_group_age_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced', max_depth=10)
fs_group_age_predict.fit([fs_f_g[j] for j in train_s_ind], group_age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, fs_group_age_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
fs_group_age = fs_group_age_predict.predict(fs_f)
fs_group_age_proba = fs_group_age_predict.predict_proba(fs_f)
t_lda_group_age_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced', max_depth=10)
t_lda_group_age_predict.fit([t_lda_f_g[j] for j in train_s_ind], group_age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, t_lda_group_age_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
t_lda_group_age = t_lda_group_age_predict.predict(t_lda_f)
t_lda_group_age_proba = t_lda_group_age_predict.predict_proba(t_lda_f)
i_group_age_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced', max_depth=10)
i_group_age_predict.fit([i_f_g[j] for j in train_s_ind], group_age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, i_group_age_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
i_group_age = i_group_age_predict.predict(i_f)
i_group_age_proba = i_group_age_predict.predict_proba(i_f)
t_h_group_age_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced', max_depth=10)
t_h_group_age_predict.fit([t_h_f_g[j] for j in train_s_ind], group_age_s_ind)
train = [l1 + l2 for (l1, l2) in zip(train, t_h_group_age_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
t_h_group_age = t_h_group_age_predict.predict(t_h_f)
t_h_group_age_proba = t_h_group_age_predict.predict_proba(t_h_f)


In [25]:
group_age_a_predict = RandomForestClassifier(max_depth=10)
group_age_a_predict.fit(train, group_age_f_ind)
group_age_a = group_age_a_predict.predict([l1 + l2 + l3 + l4 + l5 for (l1, l2, l3, l4, l5) in zip(t_liwc_group_age_proba,
                                                                                            fs_group_age_proba,
                                                                                            t_lda_group_age_proba,
                                                                                            i_group_age_proba,
                                                                                            t_h_group_age_proba)])
group_age_a_proba = group_age_a_predict.predict_proba([l1 + l2 + l3 + l4 + l5 for (l1, l2, l3, l4, l5) in zip(t_liwc_group_age_proba,
                                                                                            fs_group_age_proba,
                                                                                            t_lda_group_age_proba,
                                                                                            i_group_age_proba,
                                                                                            t_h_group_age_proba)])


## Early scores results

In [53]:
len(gender_a)
gender_a.shape

numpy.ndarray

In [46]:
len(age_a)
age_a.shape

(33413,)

In [47]:
len(group_age_a)
group_age_a.shape

(33413,)

In [56]:
len(t_liwc_f)
arr = np.array(t_liwc_f)
arr.shape
t_liwc_f

[[18.0,
  18.0,
  0.0,
  94.44,
  22.22,
  16.67,
  16.67,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  5.56,
  11.11,
  11.11,
  0.0,
  5.56,
  5.56,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  11.11,
  0.0,
  5.56,
  0.0,
  5.56,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0],
 [15.0,
  15.0,
  0.0,
  86.67,
  46.67,
  33.33,
  13.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  6.67,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  6.67,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  6.67,
  6.67,
  0.0,
  0.0,
  0.0,
  0.0,
  6.67,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  6.67,
  0.0,
  6.67,
  6.67,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  6.67,
  0.0,
  0.0,
  6.67,
  0.0,
  0.0,
  6.67,
  0.0,
  13.33,
  0.0

In [41]:
len(fs_f)

33413

In [42]:
len(t_lda_f)

33413

In [43]:
len(i_f)

33413

In [44]:
len(t_h_f)

33413

In [ ]:
for i in range(70, 155, 5):
    early_label_predict = RandomForestClassifier(n_estimators=i, class_weight='balanced')
    early_label_scores = cross_val_score(early_label_predict,
                                     [[e1] + [e2] + [e3] + l4 + l5 + l6 + l7 + l8
                                      for (e1, e2, e3, l4, l5, l6, l7, l8) in
                                      zip(gender_a, age_a, group_age_a, t_liwc_f, fs_f, t_lda_f, i_f, t_h_f)],
                                     labels, cv=10, scoring='f1_macro').mean()
    print(early_label_scores)


In [28]:
kf = KFold(n_splits=10)
feature_ind = range(len(labels))
avg_f1 = 0

In [ ]:
for train_ind, test_ind in kf.split(feature_ind, labels):
    labels_train = [labels[i] for i in train_ind]
    labels_test = [labels[i] for i in test_ind]
    train_s_ind, train_f_ind, labels_s_ind, labels_f_ind = train_test_split(train_ind, labels_train,
                                                                                  test_size=0.7,
                                                                                  stratify=labels_train,
                                                                                  random_state=1)

    labels_predict = RandomForestClassifier(n_estimators=120, class_weight='balanced', max_depth=10)
    labels_predict.fit([[e1] + [e2] + [e3] + l4 for (e1, e2, e3, l4) in zip(t_liwc_gender, t_liwc_age, t_liwc_group_age,
                                                                            [t_liwc_f[j] for j in train_s_ind])],
                       labels_s_ind)
    train = labels_predict.predict_proba([t_liwc_f_g[j] for j in train_f_ind])
    test = labels_predict.predict_proba([t_liwc_f_g[j] for j in test_ind])
    labels_predict = RandomForestClassifier(n_estimators=140, class_weight='balanced', max_depth=10)
    labels_predict.fit([[e1] + [e2] + [e3] + l4 for (e1, e2, e3, l4) in zip(t_liwc_gender, t_liwc_age, t_liwc_group_age,
                                                                            [fs_f_g[j] for j in train_s_ind])],
                       labels_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, labels_predict.predict_proba([fs_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, labels_predict.predict_proba([fs_f_g[j] for j in test_ind]))]
    labels_predict = RandomForestClassifier(n_estimators=70, class_weight='balanced', max_depth=10)
    labels_predict.fit([[e1] + [e2] + [e3] + l4 for (e1, e2, e3, l4) in zip(t_liwc_gender, t_liwc_age, t_liwc_group_age,
                                                                            [t_lda_f_g[j] for j in train_s_ind])],
                       labels_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, labels_predict.predict_proba([t_lda_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, labels_predict.predict_proba([t_lda_f_g[j] for j in test_ind]))]
    labels_predict = RandomForestClassifier(n_estimators=130, class_weight='balanced', max_depth=10)
    labels_predict.fit([[e1] + [e2] + [e3] + l4 for (e1, e2, e3, l4) in zip(t_liwc_gender, t_liwc_age, t_liwc_group_age,
                                                                            [i_f_g[j] for j in train_s_ind])],
                       labels_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, labels_predict.predict_proba([i_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, labels_predict.predict_proba([i_f_g[j] for j in test_ind]))]
    labels_predict = RandomForestClassifier(n_estimators=75, class_weight='balanced', max_depth=10)
    labels_predict.fit([[e1] + [e2] + [e3] + l4 for (e1, e2, e3, l4) in zip(t_liwc_gender, t_liwc_age, t_liwc_group_age,
                                                                            [t_h_f_g[j] for j in train_s_ind])], labels_s_ind)
    train = [l1 + l2 for (l1, l2) in zip(train, labels_predict.predict_proba([t_h_f_g[j] for j in train_f_ind]))]
    test = [l1 + l2 for (l1, l2) in zip(test, labels_predict.predict_proba([t_h_f_g[j] for j in test_ind]))]

    labels_predict = RandomForestClassifier(max_depth=10)
    labels_predict.fit(train, labels_f_ind)
    labels_pred = labels_predict.predict(test)
    avg_f1 += f1_score(labels_s_ind, labels_pred, average='macro')
